In [1]:
from pymatgen.io.vasp.outputs import Vasprun
import numpy as np
import os
import glob
from pymatgen.electronic_structure.core import Spin

In [2]:
def BZ_avg(eigenvalues, band_index,kpoints,kpoints_weights):
    """
    :param eigenvalues: eigenvalues of all bands with the format of Vasprun.eigenvalues
    :param band_index: the band index for averaging
    :param kpoints: sampled k-points
    :param kpoints_weights: the weights of sampled k-points
    :return: the average of a particular band over k-points in the BZ
    """
    energy_array = [ eigenvalues[Spin.up][k_point_index][band_index][0] for k_point_index in range(len(kpoints))]
    return np.average(energy_array, weights=kpoints_weights)

In [8]:
def Branch_point_energy(vr, cb_sampling_num, vb_sampling_num):
    """
    Calculate the Branch_point_energy with the formula proposed in:
    APPLIED PHYSICS LETTERS 94, 012104 2009 ; doi: 10.1063/1.3059569

    :param vr: Vasprun Object
    :param cb_sampling_num: the number of cb for sampling. Default vb_sampling_num = 2* cb_sampling_num
    :return:
    """
    #vb_sampling_num = 2*cb_sampling_num
    vbm_band_index = len(list(filter(lambda u:u[1]!=0, vr.eigenvalues[Spin.up][0])))-1

    vb_sampling_indexes = [vbm_band_index-i for i in range(vb_sampling_num)]
    cb_sampling_indexes = [vbm_band_index+i+1 for i in range(cb_sampling_num)]
    

    vb_avg = sum([BZ_avg(vr.eigenvalues,vb_index,vr.actual_kpoints,vr.actual_kpoints_weights)
               for vb_index in vb_sampling_indexes])/len(vb_sampling_indexes)
    cb_avg = sum([BZ_avg(vr.eigenvalues,cb_index,vr.actual_kpoints,vr.actual_kpoints_weights)
               for cb_index in cb_sampling_indexes])/len(cb_sampling_indexes)
    
    return np.mean([vb_avg,cb_avg])

In [10]:
folder = '/Users/yao/Google Drive/data/113/113-pbe-soc-hiroko'
if __name__ == "__main__":


    for compound in ['100MAPbI3', '111MAPbI3_0.01', '111MAPbI3_0.05', '111MAPbBr3_0.01', '111MAPbCl3_0.01']:              
        try:
            currentfolder = folder + '/' + compound 
            vr = Vasprun(os.path.join(currentfolder,'vasprun.xml'))
            vbm = vr.eigenvalue_band_properties[2]
            cbm = vr.eigenvalue_band_properties[1]
            ebp = Branch_point_energy(vr, cb_sampling_num=2, vb_sampling_num=4)
            bandgap = vr.eigenvalue_band_properties[0]
            if bandgap == 0:
                ebp_position = 1000
            else:
                ebp_position = (ebp-vbm)/bandgap
            print('[\''+ compound + '\',' + str(vbm) + ',' + str(ebp) + ',' + str(cbm) +'],')
                               
        except:
            print(compound)
                    

//anaconda/lib/python3.5/site-packages/pymatgen/io/vasp/outputs.py:814: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/yao/Google Drive/data/113/113-pbe-soc-hiroko/100MAPbI3
  " was found in {}".format(os.path.abspath(p)))


['100MAPbI3',1.0697,1.89945249109,1.8558],


//anaconda/lib/python3.5/site-packages/pymatgen/io/vasp/outputs.py:814: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/yao/Google Drive/data/113/113-pbe-soc-hiroko/111MAPbI3_0.01
  " was found in {}".format(os.path.abspath(p)))


['111MAPbI3_0.01',0.9606,1.73457081142,1.613],


//anaconda/lib/python3.5/site-packages/pymatgen/io/vasp/outputs.py:814: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/yao/Google Drive/data/113/113-pbe-soc-hiroko/111MAPbI3_0.05
  " was found in {}".format(os.path.abspath(p)))


['111MAPbI3_0.05',0.9696,1.74146051841,1.62],


//anaconda/lib/python3.5/site-packages/pymatgen/io/vasp/outputs.py:814: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/yao/Google Drive/data/113/113-pbe-soc-hiroko/111MAPbBr3_0.01
  " was found in {}".format(os.path.abspath(p)))
//anaconda/lib/python3.5/site-packages/pymatgen/io/vasp/outputs.py:390: UnconvergedVASPWarning: /Users/yao/Google Drive/data/113/113-pbe-soc-hiroko/111MAPbBr3_0.01/vasprun.xml is an unconverged VASP run.
Electronic convergence reached: True.
Ionic convergence reached: False.
  warnings.warn(msg, UnconvergedVASPWarning)


['111MAPbBr3_0.01',-0.1848,0.619549906086,0.8714],


//anaconda/lib/python3.5/site-packages/pymatgen/io/vasp/outputs.py:814: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/yao/Google Drive/data/113/113-pbe-soc-hiroko/111MAPbCl3_0.01
  " was found in {}".format(os.path.abspath(p)))
//anaconda/lib/python3.5/site-packages/pymatgen/io/vasp/outputs.py:390: UnconvergedVASPWarning: /Users/yao/Google Drive/data/113/113-pbe-soc-hiroko/111MAPbCl3_0.01/vasprun.xml is an unconverged VASP run.
Electronic convergence reached: True.
Ionic convergence reached: False.
  warnings.warn(msg, UnconvergedVASPWarning)


['111MAPbCl3_0.01',-0.0169,0.81879947408,1.4824],
